In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime
import util_midia

In [2]:
link = 'http://www.vermelho.org.br/ultimas.php'

In [3]:
link_site = 'http://www.vermelho.org.br/'

In [4]:
req = requests.get(link)

In [5]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('article', class_='categoria_item ')

In [6]:
for noticia in noticias:
    ref_link = noticia.find_all('a', href=True)[0]['href']
    print(noticia.find_all('a', href=True)[0]['href']) 
    print(link_site+ref_link)
    if (link_site not in ref_link):
        util_midia.social_news_from_link(link_site+ref_link)
        print(link_site+ref_link)
    else:
        util_midia.social_news_from_link(ref_link)
        print(ref_link)

noticia/316861-1
http://www.vermelho.org.br/noticia/316861-1
100% [..............................................................................] 71005 / 71005tim.php
C:\Users\admin\rss3\src\midia_postagem\images\tim.php
['EUA mantêm cerca de 200 menores migrantes separados dos pais - Portal Vermelho']
news_in_db: False
 -- no categories -- 
http://www.vermelho.org.br/noticia/316861-1
noticia/316860-1
http://www.vermelho.org.br/noticia/316860-1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100% [..............................................................................] 69707 / 69707tim.php
C:\Users\admin\rss3\src\midia_postagem\images\tim.php
['Boulos visita Lula: Moro ministro é prova da parcialidade do juiz - Portal Vermelho']
news_in_db: False
 -- no categories -- 
http://www.vermelho.org.br/noticia/316860-1
noticia/316859-1
http://www.vermelho.org.br/noticia/316859-1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100% [..............................................................................] 32107 / 32107tim.php
C:\Users\admin\rss3\src\midia_postagem\images\tim.php
['Bravatas de Bolsonaro geram incertezas']
news_in_db: False
 -- no categories -- 
http://www.vermelho.org.br/noticia/316859-1
noticia/316856-1
http://www.vermelho.org.br/noticia/316856-1


...

100% [..............................................................................] 81953 / 81953tim.php
C:\Users\admin\rss3\src\midia_postagem\images\tim.php
['Diretor da OIT critica Bolsonaro e embaixadora brasileira reclama - Portal Vermelho']
news_in_db: False
 -- no categories -- 
http://www.vermelho.org.br/noticia/316855-1
noticia/316854-1
http://www.vermelho.org.br/noticia/316854-1
100% [..............................................................................] 34316 / 34316tim.php
C:\Users\admin\rss3\src\midia_postagem\images\tim.php
['Senado reduz à metade fundo do Pré-Sal para Saúde e Educação - Portal Vermelho']
news_in_db: False
 -- no categories -- 
http://www.vermelho.org.br/noticia/316854-1
noticia/316853-1
http://www.vermelho.org.br/noticia/316853-1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100% [..............................................................................] 49813 / 49813tim.php
C:\Users\admin\rss3\src\midia_postagem\images\tim.php
['Bolsonaro ataca BNDES, e funcionários reagem']
news_in_db: False
 -- no categories -- 
http://www.vermelho.org.br/noticia/316853-1
noticia/316852-1
http://www.vermelho.org.br/noticia/316852-1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100% [..............................................................................] 42953 / 42953tim.php
C:\Users\admin\rss3\src\midia_postagem\images\tim.php
['Enquanto os bancos lucram, 62% dos brasileiros estão endividados - Portal Vermelho']
news_in_db: False
 -- no categories -- 
http://www.vermelho.org.br/noticia/316852-1


In [7]:
noticias[0]

<article class="categoria_item ">
<a href="noticia/316861-1" title="EUA mantêm cerca de 200 menores migrantes separados dos pais">
<picture alt="EUA mantêm cerca de 200 menores migrantes separados dos pais">
<source media="(min-width: 960px)" srcset="tim.php?src=http://www.vermelho.org.br/admin/arquivos/biblioteca/detencao_usa89627.jpg&amp;w=400&amp;h=225"/>
<source media="(min-width: 380px)" srcset="tim.php?src=http://www.vermelho.org.br/admin/arquivos/biblioteca/detencao_usa89627.jpg&amp;w=150&amp;h=84"/>
<source media="(min-width: 1px)" srcset="tim.php?src=http://www.vermelho.org.br/admin/arquivos/biblioteca/detencao_usa89627.jpg&amp;w=120&amp;h=110"/>
<img alt="[EUA mantêm cerca de 200 menores migrantes separados dos pais]" src="http://www.vermelho.org.br/admin/arquivos/biblioteca/detencao_usa89627.jpg" title="EUA mantêm cerca de 200 menores migrantes separados dos pais"/>
</picture>
<div class="categoria_item_desc">
<time datetime="2018-11-09 10:15:00">09/11/2018 10h15</time>
<h1>